In [ ]:
#SNN : shared Neirest Neighbors (clustering basé graphe)
# graphe de voisinage
#(DBSCAN, clustering basé densité)
# cours :https://homepages.laas.fr/huguet/drupal/sites/homepages.laas.fr.huguet/files/u78/2018-2019-cours_AP-NS-Part-2-Clustering.pdf 

# SNN et DBSCAN permet de gérer des densité de points différentes, de trouver des formes complexes

#Algorithme SNN-DBSCAN
#1. Calculer la matrice de similarité point à point
#2.Filtrer la matrice pour ne conserver que k voisins les plus similaires
#3.Construire le graphe SNN
#4.Appliquer le principe de DBSCAN
#   Paramètres : epsilon et MinPts
#     Calculer le epsilon-voisinage de chaque point  x
#     densité SNN(x)
#     Déterminer les points intérieurs (voisinage au moins de taille MinPts) et construire les clusters associés
#     Retirer les points atypiques86


# Variante : Jarvis-Patrick 
#1.Calculer la matrice de similarité point à point
#2.Filtrer la matrice pour ne conserver que k voisins les plus similaires
#3.Construire le graphe SNN
#   Appliquer un seuil de similarité sur la matrice 
#   appliquer une recherche de composantes connexes pour obtenir les clusters


#   Méthode ChameleonPré-processing : 
#   Calculer le graphe des plus proches voisins
#   Deux phases
#   Déterminer des sous clusters initiaux (partition du graphe des k-ppv)
#   Approche hiérarchique ascendante : 
        #Mesures spécifiques pour regrouper des clusters (inter-connectivité et proximité)88


In [16]:
# jeu de données
# matrice de distance
# matrice de voisinage
# DBSCAN
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean





In [26]:
# 1. CONSTRUIRE LA MATRICE DE SIMILARITE
# SIMILARITE : 1/(1+distance)

data1 = np.loadtxt( 'cham-data/t4.8k.dat' )
# Autre possibilité, genloadstxt
data1[1][0]
print(' shape data1 : ' , data1.shape)
print(data1[1,])
# chaque ligne représente les coordonnées d'un point


def similarity_func(u, v):
    return 1/(1+euclidean(u,v))

def similarity_Matrix(data):
    numPoints = len(data)
    matrix = np.zeros(shape=(numPoints, numPoints))
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            sim = similarity_func(data[i], data[j])
            matrix[i,j] = sim
    return matrix

# Similarity matrix OK

 shape data1 :  (8000, 2)
[454.665985 264.80899 ]


In [43]:
#test Similarity_Matrix
data=([
    [1, 0],
    [1, 1],
    [2, 1]])
#similarity_Matrix(data)
sim1=similarity_Matrix(data1[1:5,:])

In [44]:
sim1


array([[1.        , 0.00272431, 0.01203834, 0.00373232],
       [0.00272431, 1.        , 0.00347096, 0.00424803],
       [0.01203834, 0.00347096, 1.        , 0.00436448],
       [0.00373232, 0.00424803, 0.00436448, 1.        ]])

In [68]:
# ICI, le graphe des kneibors nous permet de faire l'étape 1 et l'etape 2 en même temps.
# par défaut, il compare les distances euclidiennes, mais on peut la setter.
#DOC ICI : https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.kneighbors_graph.html
#MJ dit : c'est OK pour l'utiliser. le but étant de faire un meilleur score que DBSCAN
from sklearn.neighbors import kneighbors_graph
A = kneighbors_graph(sim1, 2, mode='connectivity', include_self= False)

#TODO : trouver les paramètres optimaux et comment appliquer la suite de l'algo.
# idée : se servir de A pour mettre à jour la matrice de similarité
print(A)

  (0, 2)	1.0
  (0, 3)	1.0
  (1, 3)	1.0
  (1, 2)	1.0
  (2, 0)	1.0
  (2, 3)	1.0
  (3, 2)	1.0
  (3, 1)	1.0


In [52]:
#2.Filtrer la matrice pour ne conserver que k voisins les plus similaires
#Method to compute K-Nearest Neighbors of points
def SparseSimMatrix(simMatrix, k):
    count=len(simMatrix)
    # matrix = [[0 for i in range(count)] for j in range(count)]
    #matrix with dimension count*count
    for i in range(count):
        matrixsorted = sorted(simMatrix[i], key=lambda x: x[1])# sort each row of Matrix based on distance(value)
        #print matrixsorted
        for j in range(k):
            #file.write(str(matrixsorted[0][j+1]) + " ")# write all the k nearest neighbors to file
            similarityMatrix[i][j] = int(matrixsorted[j+1][0][1])#Assign each row  in similarityMatrix to all the k nearest neighbors..
    return similarityMatrix

In [65]:
sorted(sim1[0] )

[0.002724314270051647, 0.0037323236050198698, 0.012038337656274696, 1.0]